scratch

In [98]:
import os
folder = './data/'
def docs():
    for file_name in os.listdir(folder):
        with open(folder + file_name, "r", encoding='utf-8') as text_file:
            #read file to a string & split
            yield text_file.read()

In [99]:
#!pip3 install -U spacy

#!python3 -m spacy download en_core_web_lg
#!python -m spacy download en_core_web_md
#!python -m spacy download en_core_web_sm

#!pip install spacy-lookups-data
#!python3 -m spacy validate

In [110]:
import spacy
import re
nlp = spacy.load("en_core_web_sm")
nlp = spacy.load("en_core_web_lg")
# for i, doc in enumerate(docs()):
#     tokens = nlp(doc)
#     for j,token in enumerate(tokens):
#         if i <10 and j<10:
#             print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)

In [111]:
sample_text = """
Method for optimizing word classification in machine learning text 

The invention relates to the field of data processing and machine learning classification, in particular to a method for optimizing word classification in machine learning text. The method comprises the steps that on the basis of text classification, self-defined and semantically related features are filtered out through a feature selection regulator based on a regular expression, a user customizes classification types in training data after feature selection, and classification based training is conducted by means of the features and the types according to a naive Bayesian model; after training is completed, in the application stage, if statements conforming to the feature selection regulator exist in text needing word classification, classification is completed by combining a trained model. According to the method, the capacity of the model for processing work classification is not limited in word data in a training sample; the method can be applied to optimization and application of machine learning text work classification and derivation functions thereof."""

In [116]:
def spacy_tokenizer_lemmatizer(text,debug = False):
    
    doc = nlp(text)
    #tokens = tokenizer(text)
    
    lemma_list = []
    for token in doc:
        if re.match('\s+',token.text):
            if debug: 
                print('skipping ws')
            continue    
        elif token.is_stop is False:
            lemma_list.append(token.lemma_)             
            if token.text != token.lemma_:
                if debug: print(f'+ {token.lemma_} <- {token.text}')
                     
            else:
                if debug: print(f'+ {token.lemma_}')
            continue
        else:
             if debug: print(f'- {token.lemma_}')
    return set(lemma_list)

spacy_tokenizer_lemmatizer(sample_text)

{',',
 '-',
 '.',
 ';',
 'Method',
 'accord',
 'application',
 'apply',
 'base',
 'basis',
 'bayesian',
 'capacity',
 'classification',
 'combine',
 'complete',
 'comprise',
 'conduct',
 'conform',
 'customize',
 'datum',
 'define',
 'derivation',
 'exist',
 'expression',
 'feature',
 'field',
 'filter',
 'function',
 'invention',
 'learn',
 'learning',
 'limit',
 'machine',
 'mean',
 'method',
 'model',
 'naive',
 'need',
 'optimization',
 'optimize',
 'particular',
 'process',
 'processing',
 'regular',
 'regulator',
 'relate',
 'related',
 'sample',
 'selection',
 'self',
 'semantically',
 'stage',
 'statement',
 'step',
 'text',
 'thereof',
 'train',
 'training',
 'type',
 'user',
 'word',
 'work'}

In [68]:
tokenize = lambda data: word_tokenize(str(data))
to_lower = lambda texts: [x.lower() for x in texts]
dehyphenate = lambda tokens: [part for token in tokens for part in token.split('-')]

xforms = {"ca": 'can', "n't": 'not', "'s": ''}
xform_map = lambda tokens, xforms: (xforms[token] if token in xforms else token for token in tokens)

stop_words = stopwords.words('english')
stop_word_filter = lambda tokens, stop: (token for token in tokens if token not in stop)

stemmer = PorterStemmer()
stem = lambda tokens,stemmer : (stemmer.stem(token) for token in tokens)

print(string.punctuation)
table = str.maketrans('', '', string.punctuation)
punctuation_filter = lambda data, table: (w.translate(table) for w in data)

len_filter = lambda tokens, n : (token for token in tokens if len(token) > n )

def preprocess(doc:str) -> set:
    """
    preprocessing pipeline
    takes for a document
    might be better converting puntuation to tokens like , to COMMA '''
    """
    
    tokens = tokenize(doc)
    tokens = dehyphenate(tokens)
    tokens = to_lower(tokens)
    tokens = xform_map(tokens,xforms)
    tokens = stop_word_filter(tokens,stop_words)
    tokens = stem(tokens,stemmer)
    tokens = punctuation_filter(tokens,table)
    tokens = len_filter(tokens,1)
    return tokens

def preprocess_spacy(doc:str)-> set:
    """
    preprocessing pipeline with better handling of stop words
    """
    tokens = spacy_tokenizer_lemmatizer(doc)
    tokens = dehyphenate(tokens)
    tokens = to_lower(tokens)
    tokens = xform_map(tokens,xforms)
    tokens = stem(tokens,stemmer)
    tokens = punctuation_filter(tokens,table)
    tokens = len_filter(tokens,1)
    return tokens


print(preprocess(sample_text))

NameError: name 'stopwords' is not defined

Apple Apple PROPN NNP nsubj Xxxxx True False
is be AUX VBZ aux xx True True
looking look VERB VBG ROOT xxxx True False
at at ADP IN prep xx True True
buying buy VERB VBG pcomp xxxx True False
U.K. U.K. PROPN NNP compound X.X. False False
startup startup NOUN NN dobj xxxx True False
for for ADP IN prep xxx True True
$ $ SYM $ quantmod $ False False
1 1 NUM CD compound d False False
billion billion NUM CD pobj xxxx True False


In [ ]:
from tqdm.notebook import tqdm

def buildVocabulary(pipline,vocabulary):
    """
    build vocabulary
    """ 
    #open text file in read mode 
    doc_bar=tqdm(docs())
    for i,doc in enumerate(doc_bar):
        #read whole file to a string
        words = pipline(doc)
        vocabulary.update(words)
        doc_bar.set_description(f"vocab size = {len(vocabulary)} docs= {i+1}")
    return vocabulary